Team Members:

Ch Gnaneshwar: cs24mtech11015

Saswata Mishra: cs24mtech12001

# Assignment: 5
## Problem statement:
Implement algorithm to find minimum weight perfect matching in a edge weighted bipartite graph, you need to implement the method discussed in class.

Input: CSV file with 3 rows and number of columns is number of edges.
            Each column corresponds to a edge.
            First and second rows are vertex ids of the edges, and third row is the edge weights. Note that vertex ids are integers.
              
Output:  You need to print values of dual variables at each iteration.

Instructions: 1. You need to implement in google Collab and share the link
                       2. Max two members in a group. You need to mention roll number of group members at the top of the code

## 1. Imports and Edge Class Definition

In [1]:
import csv
import math
import copy
import sys
from collections import deque, defaultdict, namedtuple

# Edge structure
class Edge:
    def __init__(self, u, v, cap, cost, eid):
        self.u = int(u)
        self.v = int(v)
        self.cap = float(cap)
        self.cost = float(cost)
        self.flow = 0.0
        self.id = int(eid)
    
    def __repr__(self):
        return f"Edge(id={self.id}, {self.u}->{self.v}, cap={self.cap}, cost={self.cost}, flow={self.flow})"

## 2. CSV Input Reader

Reads network data from CSV file with 4 rows:
- Row 1: from_nodes
- Row 2: to_nodes
- Row 3: capacities
- Row 4: costs

In [2]:
def read_csv_input(filename):
    rows = []
    with open(filename, newline='') as f:
        r = csv.reader(f)
        for row in r:
            if len(row) == 0:
                continue
            rows.append(row)
    
    if len(rows) != 4:
        raise ValueError("CSV must have exactly 4 non-empty rows: from, to, capacity, cost")
    
    ncols = len(rows[0])
    edges = []
    nodes = set()
    
    for j in range(ncols):
        u = int(rows[0][j])
        v = int(rows[1][j])
        cap = float(rows[2][j])
        cost = float(rows[3][j])
        e = Edge(u, v, cap, cost, j)
        edges.append(e)
        nodes.add(u)
        nodes.add(v)
    
    return sorted(nodes), edges

## 3. Edmonds-Karp Algorithm (Max-Flow)

Uses BFS to find augmenting paths in the residual graph.
- Maintains forward edges (original capacity)
- Maintains backward edges (residual capacity from flow sent)

In [3]:
def edmonds_karp(nodes, edges, source, sink):
    # Build adjacency graph with forward/back entries
    graph = defaultdict(list)
    
    # Create explicit forward/back edge entries with 'rev' pointers
    for e in edges:
        # forward
        fwd = {'to': e.v, 'cap': e.cap, 'eid': e.id, 'rev': None}
        bwd = {'to': e.u, 'cap': 0.0, 'eid': e.id, 'rev': None}
        fwd['rev'] = bwd
        bwd['rev'] = fwd
        graph[e.u].append(fwd)
        graph[e.v].append(bwd)
    
    flow = 0.0
    parent = {}
    
    while True:
        # BFS to find augmenting path
        q = deque([source])
        parent = {source: None}
        parent_edge = {}
        found = False
        
        while q and not found:
            u = q.popleft()
            for edge in graph[u]:
                if edge['cap'] > 1e-12 and edge['to'] not in parent:
                    parent[edge['to']] = u
                    parent_edge[edge['to']] = edge
                    if edge['to'] == sink:
                        found = True
                        break
                    q.append(edge['to'])
        
        if not found:
            break
        
        # Find bottleneck
        v = sink
        bottleneck = float('inf')
        while v != source:
            e = parent_edge[v]
            bottleneck = min(bottleneck, e['cap'])
            v = parent[v]
        
        # Augment flow
        v = sink
        while v != source:
            e = parent_edge[v]
            e['cap'] -= bottleneck
            e['rev']['cap'] += bottleneck
            v = parent[v]
        
        flow += bottleneck
    
    # Extract flow per original edge
    eid_to_flow = {}
    for e in edges:
        # Find forward entry in graph[e.u] with eid e.id
        fwd = None
        for ent in graph[e.u]:
            if ent['eid'] == e.id and ent['to'] == e.v:
                fwd = ent
                break
        
        if fwd is None:
            eid_to_flow[e.id] = 0.0
        else:
            used = max(0.0, e.cap - fwd['cap'])
            eid_to_flow[e.id] = used
    
    # Assign flows to edges list
    for e in edges:
        e.flow = eid_to_flow[e.id]
    
    return flow

## 4. Helper Functions for Network Simplex

Utility functions for:
- Building edge mappings
- Creating adjacency lists from spanning tree
- Finding paths in spanning tree

In [4]:
def build_edges_map(edges):
    return {e.id: e for e in edges}

def adjacency_from_tree(tree_arcs, edges_map):
    adj = defaultdict(list)
    for eid in tree_arcs:
        e = edges_map[eid]
        adj[e.u].append((e.v, e))
        adj[e.v].append((e.u, e))
    return adj

def find_path_in_tree(tree_arcs, edges_map, start, end):
    """Find path from start to end in the spanning tree using BFS"""
    adj = adjacency_from_tree(tree_arcs, edges_map)
    q = deque([start])
    parent = {start: None}
    parent_edge = {}
    
    while q:
        x = q.popleft()
        if x == end:
            break
        for (nbr, e) in adj.get(x, []):
            if nbr not in parent:
                parent[nbr] = x
                parent_edge[nbr] = e
                q.append(nbr)
    
    if end not in parent:
        return []
    
    # Reconstruct path with directions
    path = []
    cur = end
    while parent[cur] is not None:
        e = parent_edge[cur]
        if e.u == parent[cur] and e.v == cur:
            dir_on_path = +1
        else:
            dir_on_path = -1
        path.append((e, dir_on_path))
        cur = parent[cur]
    
    path.reverse()
    return path

## 5. Node Potentials and Reduced Costs

- **Node potentials (π)**: Dual variables computed from spanning tree
- **Reduced cost**: c_ij + π_i - π_j (measures optimality)

In [5]:
def recompute_potentials(nodes, edges, tree_arcs, root=None):
    """Compute node potentials using BFS from root in spanning tree"""
    edges_map = build_edges_map(edges)
    if root is None:
        root = min(nodes)
    
    pi = {n: None for n in nodes}
    pi[root] = 0.0
    adj = adjacency_from_tree(tree_arcs, edges_map)
    q = deque([root])
    
    while q:
        x = q.popleft()
        for (nbr, e) in adj.get(x, []):
            if pi[nbr] is not None:
                continue
            if e.u == x and e.v == nbr:
                pi[nbr] = pi[x] + e.cost
            else:
                pi[nbr] = pi[x] - e.cost
            q.append(nbr)
    
    for n in nodes:
        if pi[n] is None:
            pi[n] = 0.0
    
    return pi

def compute_reduced_costs(edges, pi):
    """Compute reduced cost for each edge: c_ij + pi_i - pi_j"""
    r = {}
    for e in edges:
        r[e.id] = e.cost + pi[e.u] - pi[e.v]
    return r

## 6. Entering Arc Selection (Bland's Rule)

Selects the non-tree arc with:
1. Most negative reduced cost
2. Smallest edge ID (tie-breaker for anti-cycling)

In [6]:
def select_entering_arc(edges, reduced_costs, tree_arcs, tol=1e-12):
    """Select entering arc using Bland's rule (smallest id among most negative reduced cost)"""
    candidates = []
    
    for e in edges:
        if e.id in tree_arcs:
            continue
        
        # Check residual availability
        res_fwd = e.cap - e.flow  # forward direction
        res_bwd = e.flow          # backward direction
        r = reduced_costs[e.id]
        
        # Forward orientation reduces objective if r < -tol and has capacity
        if res_fwd > tol and r < -tol:
            candidates.append((r, e.id, e, +1))
        
        # Backward orientation reduces objective if -r < -tol and res_bwd > tol
        if res_bwd > tol and -r < -tol:
            candidates.append((-r, e.id, e, -1))
    
    if not candidates:
        return None
    
    # Sort by reduced cost, then by edge id (Bland's rule)
    candidates.sort(key=lambda x: (x[0], x[1]))
    _, _, edge, direction = candidates[0]
    return (edge, direction)

## 7. Cycle Formation and Augmentation

- Form cycle by adding entering arc to tree path
- Compute theta (bottleneck capacity)
- Find leaving arc
- Augment flow along cycle

In [7]:
def find_cycle(tree_arcs, edges_map, entering_edge, enter_dir=+1):
    """Form cycle by adding entering edge to tree path from u to v"""
    path = find_path_in_tree(tree_arcs, edges_map, entering_edge.u, entering_edge.v)
    cycle = list(path)
    cycle.append((entering_edge, enter_dir))
    return cycle

def compute_theta_and_leaving(cycle, tol=1e-12):
    """Compute bottleneck capacity (theta) and identify leaving arc"""
    theta = float('inf')
    leaving = None
    leaving_dir = None
    
    for (e, dir) in cycle:
        if dir == +1:
            avail = e.cap - e.flow
        else:
            avail = e.flow
        
        if avail < theta - 1e-15:
            theta = avail
            leaving = e
            leaving_dir = dir
    
    if theta == float('inf'):
        theta = 0.0
    if theta < tol:
        theta = 0.0
    
    return theta, leaving, leaving_dir

def augment_cycle(cycle, theta):
    """Augment flow by theta along the cycle"""
    for (e, dir) in cycle:
        if dir == +1:
            e.flow += theta
        else:
            e.flow -= theta

## 8. Initial Spanning Tree Construction

In [8]:
def build_initial_tree_from_flow(nodes, edges, source):
    """Build initial spanning tree using BFS (|V|-1 edges)"""
    edges_map = build_edges_map(edges)
    adj = defaultdict(list)
    
    for e in edges:
        adj[e.u].append((e.v, e))
        adj[e.v].append((e.u, e))
    
    start = source if source in nodes else nodes[0]
    visited = set([start])
    q = deque([start])
    tree_arcs = set()
    parent = {start: None}
    
    while q:
        x = q.popleft()
        for (nbr, e) in adj.get(x, []):
            if nbr not in visited:
                visited.add(nbr)
                parent[nbr] = x
                tree_arcs.add(e.id)
                q.append(nbr)
    
    # Ensure tree has |V|-1 arcs
    i = 0
    for e in edges:
        if len(tree_arcs) >= len(nodes) - 1:
            break
        tree_arcs.add(e.id)
    
    return tree_arcs

## 9. Main Network Simplex Algorithm

Minimizes cost while maintaining fixed flow value using:
- Spanning tree basis pivots
- Bland's rule for anti-cycling (degeneracy handling)
- Reduced cost optimality check

In [9]:
def network_simplex_min_cost_fixed_flow(nodes, edges, source, sink, verbose=True, max_iters=10000):
    """Main network simplex driver for minimizing cost with fixed flow"""
    edges_map = build_edges_map(edges)
    
    # Build initial spanning tree
    tree_arcs = build_initial_tree_from_flow(nodes, edges, source)
    
    # Ensure tree has exactly |V|-1 arcs
    idx = 0
    while len(tree_arcs) < len(nodes) - 1 and idx < len(edges):
        tree_arcs.add(edges[idx].id)
        idx += 1
    
    iter_count = 0
    last_basis = None
    basis_repeat_count = 0
    
    while True:
        iter_count += 1
        if iter_count > max_iters:
            if verbose:
                print("Reached max iterations in network-simplex")
            break
        
        # Check for basis cycling (anti-cycling detection)
        current_basis = frozenset(tree_arcs)
        if current_basis == last_basis:
            basis_repeat_count += 1
            if basis_repeat_count > 10:
                if verbose:
                    print(f"Basis cycling detected after {iter_count} iterations; terminating")
                break
        else:
            basis_repeat_count = 0
            last_basis = current_basis
        
        # Compute node potentials
        pi = recompute_potentials(nodes, edges, tree_arcs, root=source)
        
        # Compute reduced costs
        reduced = compute_reduced_costs(edges, pi)
        
        # Select entering arc
        entering = select_entering_arc(edges, reduced, tree_arcs)
        if entering is None:
            if verbose:
                print("Network simplex optimality reached. Iterations:", iter_count - 1)
            break
        
        e_enter, orientation = entering
        
        # Form cycle
        cycle = find_cycle(tree_arcs, edges_map, e_enter, enter_dir=orientation)
        
        # Compute theta and leaving arc
        theta, leaving, leaving_dir = compute_theta_and_leaving(cycle)
        
        # Handle degeneracy using Bland's rule
        if theta < 1e-9:  # degenerate pivot
            # Among all blocking arcs in the cycle that are in the tree,
            # choose the one with smallest id
            blocking_arcs = []
            for (e, d) in cycle:
                if d == +1:
                    avail = e.cap - e.flow
                else:
                    avail = e.flow
                if avail < 1e-9 and e.id in tree_arcs:
                    blocking_arcs.append(e)
            
            if blocking_arcs:
                leaving = min(blocking_arcs, key=lambda x: x.id)
            elif leaving is None or leaving.id not in tree_arcs:
                # Must select a tree arc to leave
                tree_arcs_in_cycle = [e for (e, d) in cycle if e.id in tree_arcs]
                if tree_arcs_in_cycle:
                    leaving = min(tree_arcs_in_cycle, key=lambda x: x.id)
                else:
                    if verbose:
                        print("No valid leaving arc; stopping")
                    break
            theta = 0.0
        
        # Perform augmentation
        augment_cycle(cycle, theta)
        
        # Update tree: replace leaving by entering
        if leaving is not None and leaving.id in tree_arcs:
            tree_arcs.remove(leaving.id)
        tree_arcs.add(e_enter.id)
        
        if verbose:
            print(f"Iter {iter_count}: total_cost = {sum(e.flow * e.cost for e in edges):.6f}")
    
    final_cost = sum(e.flow * e.cost for e in edges)
    return edges, tree_arcs, final_cost, iter_count - 1

## 10. Main Solver Function

Combines Edmonds-Karp (max-flow) and Network Simplex (min-cost)

In [10]:
def solve_min_cost_max_flow_via_network_simplex(filename, source=1, sink=2, verbose=True):
    """Solve min-cost max-flow problem"""
    nodes, edges = read_csv_input(filename)
    
    # Create copies for Edmonds-Karp (mutates flows)
    edges_for_flow = [Edge(e.u, e.v, e.cap, e.cost, e.id) for e in edges]
    
    # Phase 1: Find max-flow
    F = edmonds_karp(nodes, edges_for_flow, source, sink)
    if verbose:
        print("Max-flow value found by Edmonds-Karp:", F)
    
    # Transfer flows to original edges
    eid_to_flow = {e.id: e.flow for e in edges_for_flow}
    for e in edges:
        e.flow = eid_to_flow.get(e.id, 0.0)
    
    # Phase 2: Minimize cost using network simplex
    edges_after, tree, final_cost, iters = network_simplex_min_cost_fixed_flow(
        nodes, edges, source, sink, verbose=verbose
    )
    
    if verbose:
        print("Final max-flow (into sink):", sum(e.flow for e in edges if e.v == sink))
        print("Final min-cost:", final_cost)
    
    return edges_after, final_cost, F

## 11. Run Solver on Test Case

In [11]:
# Configure test case
csv_path = "./testcase_with_degeneracy.csv"
print("Running solver on:", csv_path)

# Solve (source=1, sink=2 as per problem requirement)
edges_res, final_cost, F = solve_min_cost_max_flow_via_network_simplex(
    csv_path, source=1, sink=2, verbose=True
)

Running solver on: ./testcase_with_degeneracy.csv
Max-flow value found by Edmonds-Karp: 40.0
Iter 1: total_cost = 240.000000
Iter 2: total_cost = 240.000000
Iter 3: total_cost = 240.000000
Iter 4: total_cost = 240.000000
Iter 5: total_cost = 240.000000
Network simplex optimality reached. Iterations: 5
Final max-flow (into sink): 40.0
Final min-cost: 240.0


## 12. Display Results

In [12]:
print("\nFinal edge flows:")
for e in sorted(edges_res, key=lambda x: x.id):
    print(e)


Final edge flows:
Edge(id=0, 1->3, cap=10.0, cost=2.0, flow=10.0)
Edge(id=1, 1->4, cap=10.0, cost=3.0, flow=10.0)
Edge(id=2, 1->5, cap=10.0, cost=4.0, flow=10.0)
Edge(id=3, 1->6, cap=10.0, cost=5.0, flow=10.0)
Edge(id=4, 3->2, cap=10.0, cost=1.0, flow=10.0)
Edge(id=5, 3->4, cap=5.0, cost=3.0, flow=0.0)
Edge(id=6, 3->7, cap=5.0, cost=2.0, flow=0.0)
Edge(id=7, 4->2, cap=10.0, cost=2.0, flow=10.0)
Edge(id=8, 4->5, cap=5.0, cost=4.0, flow=0.0)
Edge(id=9, 4->7, cap=5.0, cost=3.0, flow=0.0)
Edge(id=10, 5->2, cap=10.0, cost=3.0, flow=10.0)
Edge(id=11, 5->6, cap=5.0, cost=5.0, flow=0.0)
Edge(id=12, 5->7, cap=5.0, cost=4.0, flow=0.0)
Edge(id=13, 6->2, cap=10.0, cost=4.0, flow=10.0)
Edge(id=14, 6->3, cap=10.0, cost=6.0, flow=0.0)
Edge(id=15, 6->4, cap=10.0, cost=7.0, flow=0.0)


## 13. Save Results to CSV

In [13]:
out_csv = "./good_testcase_with_flows.csv"

with open(out_csv, "w", newline='') as f:
    w = csv.writer(f)
    w.writerow([e.u for e in edges_res])
    w.writerow([e.v for e in edges_res])
    w.writerow([int(e.cap) if e.cap.is_integer() else e.cap for e in edges_res])
    w.writerow([e.cost for e in edges_res])
    w.writerow([e.flow for e in edges_res])

print("\nSaved output with flows to:", out_csv)


Saved output with flows to: ./good_testcase_with_flows.csv
